In [1]:
import numpy as np
import pandas as pd
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import librosa
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm,tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm,tnrange

# Data Preparation

In [2]:
# A_train = A[(A[' material']=='train')& (A[' corpus']=='Switchboard')].head(500)
# A_train.reset_index(drop=True,inplace=True)
# A_test = A[(A[' material']=='train')& (A[' corpus']=='Switchboard')].tail(100)
# A_test[' material'] = 'dev'
# A_test.reset_index(drop=True,inplace=True)
# A_final = A_train.append(A_test)
# A_final.reset_index(drop=True,inplace=True)
# A_final.to_csv('./sample_data.csv',index=False)

In [3]:
A_final = pd.read_csv('/home/dgbrizan/project/CALS_tools/file_listing.csv')
# A_final = A_final[(A_final[' material']=='train')& (A_final[' corpus']=='Switchboard')]

/data/ssubrahmannian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
A_final = A_final[(A_final[' corpus']==' Switchboard')&(A_final[' material']==' train')]

In [5]:
speakerList = A_final[' speaker'].unique().tolist()
speakerList_train = speakerList[:175]
speakerList_dev = speakerList[175:]

In [6]:
A_final.loc[A_final[' speaker'].isin(speakerList_train),' material'] = 'train'
A_final.loc[A_final[' speaker'].isin(speakerList_dev),' material'] = 'dev'

In [7]:
A_final.groupby([' material',' gender']).size()


 material   gender
dev         female    18644
            male      18804
train       female    65872
            male      48967
dtype: int64

In [8]:
A_final.reset_index(drop=True,inplace=True)

In [9]:
A_final.to_csv('./switchboardData.csv',index=False)

In [10]:
# create a sample 
B = pd.read_csv('./switchboardData.csv')
B_train = B[B[' material']=='train'].reset_index(drop=True)
B_train_sample = B_train.groupby(' gender').apply(lambda x: x.sample(500)).reset_index(drop=True)
B_dev = B[B[' material']=='dev'].reset_index(drop=True)
B_dev_sample = B_dev.groupby(' gender').apply(lambda x: x.sample(200)).reset_index(drop=True)
B_final_sample = B_train_sample.append(B_dev_sample,ignore_index=True)
B_final_sample.to_csv('./switchboardDataSample.csv',index=False)

In [11]:
class swbd_data(Dataset):
    
    def __init__(self,dataset,label,feat_len=40,paddedlen=660,encoder=None):
        
        self.dataset = dataset
        self.feat_len = feat_len
        self.paddedlen = paddedlen
        self.file_descriptions = None
        self.__longest_vector__ = 0
        self.encoder = encoder
        self.instance_label = None
        self.instance_list = []
        self.label_column = label
#         self._load_features_and_labels()
        
        
    def _get_file_address(self):
        
        type_dict = {'file': 'O',
             ' material': 'category',
             ' speaker': 'category',
             ' generation_description': 'category',
             ' generation_decade': 'category',
             ' gender': 'category',
             ' region': 'category',
             ' year': 'category',
             ' L1': 'category',
             ' corpus': 'category',
             ' education': 'category'}
#         filename = '/home/dgbrizan/project/CALS_tools/file_listing.csv'
        filename = './switchboardDataSample.csv'
        df = pd.read_csv(filename,dtype=type_dict)
        df[df.columns] = df.apply(lambda x: x.str.strip())
        final_df = df[(df[' material']==self.dataset)&(df[' corpus']=='Switchboard')].reset_index(drop=True)
        return final_df
 

    def __get_melfilter_bank_matrix__(self,file_name):
        
        # Parameters
        n_fft   = 512             # FFT size of 512 
        win_len = 512             # window size of 512 
        hop_len = 160             # more than 50% overlap while doing FFT
        n_mels  = self.feat_len   # applying 40 filters on Mel-scale to the power spectrum to extract frequency bands
        htk     = True            # Use f = 700(10m/2595−1) to convert Hz to mel 
        upper_f = 4000
        lower_f = 0
        window  = sp.signal.hamming(n_fft, sym=True) # window function used to make the wav file suitable for STFT  
    
        wav, sr = librosa.load(file_name, sr=None, mono=True)                       # Step-1: load the wav file
        magnitude_spectrogram = (1/n_fft)*np.abs(librosa.stft(wav, n_fft=n_fft, 
                                                              win_length=win_len,
                                                              hop_length=hop_len, 
                                                              window=window)) ** 2  # Step-2: Create powerspectrum
        mel_basis    = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels,
                                        fmin=lower_f, fmax=upper_f, htk=htk)        # Step-3: Filters
        mel_spectrum = np.dot(mel_basis, magnitude_spectrogram)                     # Step-4: get melspectrum
        mel_spectrum = np.where(mel_spectrum== 0, np.finfo(float).eps, mel_spectrum)# Numerical Stability
        mel_spectrum = 20 * np.log10(mel_spectrum)                                  # dB
        mel_spectrum-= (np.mean(mel_spectrum, axis=0)) # mean normalization to improve Signal to Noise ratio
        return mel_spectrum
 

    def _load_features_and_labels(self):
        
        self.file_descriptions = self._get_file_address()
        file_label = []
        padded = []  
        file_address_list = self.file_descriptions['file'].values
        
        for description in tqdm_notebook(file_address_list):
            unpadded = self.__get_melfilter_bank_matrix__(description)
            if len(unpadded[0]) > self.__longest_vector__:
                self.__longest_vector__ = len(unpadded[0])
            self.instance_list.append(unpadded)
        
        if self.paddedlen > self.__longest_vector__:
            self.paddedlen = self.__longest_vector__
            print('Final padded length truncated to longest vector')
        
        for i,instance in enumerate(self.instance_list):
            if instance.shape[1]> self.paddedlen:
                chopped_sample = self._chop_samples(instance,20)
                rand_int  = np.random.randint(0,20)
                instance = chopped_sample[rand_int]
                
            if instance.shape[1]<self.paddedlen:
                instance = self._pad_samples(instance)
                
            padded.append(instance)
            file_label.append(self.file_descriptions.loc[i,self.label_column])
        
        self.instance_list = padded
        
        if self.encoder == None:
            self.encoder = LabelEncoder()
            self.encoder.fit(file_label)
            self.instance_label = self.encoder.transform(file_label).flatten()
            return self.encoder
        
        else:
            self.instance_label = self.encoder.transform(file_label).flatten()  
        
    
    def _chop_samples(self,unaligned,num=20):
        
        padded_augments =[]
        for i in range(num):
            beg = np.random.randint(0, unaligned.shape[1] - self.paddedlen)
            padded_augments.append(unaligned[:,beg: beg + self.paddedlen])
        return padded_augments
  

    def _pad_samples(self,unpadded):
    
        if unpadded.shape[1]<self.paddedlen:
            return np.pad(unpadded,pad_width=[(0,0),(0,self.paddedlen-unpadded.shape[1])],
                          mode='constant',constant_values=0)

        
    def __getitem__(self, index):
        mel_spect = self.instance_list[index]
        mel_spect = torch.Tensor(mel_spect).unsqueeze(0)
    
        return mel_spect, self.instance_label[index]


    def __len__(self):
        return len(self.instance_list)


In [12]:
train_ds = swbd_data('train',' gender')
encoder = train_ds._load_features_and_labels()

In [13]:
val_ds = swbd_data('dev',' gender',encoder =encoder)
val_ds._load_features_and_labels()

In [14]:
train_ds[0][0].shape

torch.Size([1, 40, 660])

In [15]:
batch_size  = 138
train_dl = DataLoader(train_ds, batch_size=batch_size,shuffle=True)
vld_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

In [16]:
next(iter(train_dl))[0].shape

torch.Size([138, 1, 40, 660])

## CNN architecture 

In [17]:
    def get_file_address():
        
        type_dict = {'file': 'O',
             ' material': 'category',
             ' speaker': 'category',
             ' generation_description': 'category',
             ' generation_decade': 'category',
             ' gender': 'category',
             ' region': 'category',
             ' year': 'category',
             ' L1': 'category',
             ' corpus': 'category',
             ' education': 'category'}
        filename = '/home/dgbrizan/project/CALS_tools/file_listing.csv'
        df = pd.read_csv(filename,dtype=type_dict,nrows=5000)
        df[df.columns] = df.apply(lambda x: x.str.strip())
        return df

In [18]:
class CNN(nn.Module):
    
    def __init__(self):
        
        super(CNN, self).__init__()
        self.batchnorm_11 = nn.BatchNorm2d(1)
        self.conv2d_11 = nn.Conv2d(1,8,2)
        self.conv2d_21 = nn.Conv2d(8,16,2)
        self.mp1 = nn.MaxPool2d((2,2))
        self.dp1 = nn.Dropout(p=0.2)   
        
        self.batchnrm_f1 = nn.BatchNorm1d(100016)
        self.lin1 = nn.Linear(100016, 10000)
        self.lin2 = nn.Linear(10000, 1000)
        self.lin3 = nn.Linear(1000, 256)
        self.lin4 = nn.Linear(256, 32)  
        self.lin5 = nn.Linear(32,1)
    def forward(self, spect):
        x = F.relu(self.conv2d_11(spect))
        x = F.relu(self.conv2d_21(x))
        x = self.mp1(x)
        x = self.dp1(x)
        x = x.view(x.shape[0], -1)
        x = self.batchnrm_f1(x)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = F.relu(self.lin4(x))
        x = F.sigmoid(self.lin5(x))
        return x 

In [ ]:
def train_model(model, optim, train_dl=train_dl, verbose=False, unsqueeze =False,gpu=False):

    # Setting model in train mode
    model.train()
    total = 0
    sum_loss = 0
    for x_cat,price in tqdm(train_dl,desc='Training',dynamic_ncols=True):

        batch = price.shape[0]
        price = price.unsqueeze(1)
        price = price.float()
        if gpu:
            x_cat = x_cat.cuda()
            price = price.cuda()
        y_hat = model(x_cat)
        if unsqueeze: y_hat = y_hat.unsqueeze(1)
        loss = F.binary_cross_entropy_with_logits(y_hat,price)
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
        if verbose: print(np.sqrt(sum_loss/total))
    return sum_loss/total

def val_loss(model, valid_dl,unsqueeze=False,gpu= False):
    model.eval()
    total = 0
    sum_loss = 0
    for x_cat,price in tqdm(valid_dl,desc='Validation',dynamic_ncols=True):
        batch = price.shape[0]
        price = price.unsqueeze(1)
        price = price.float()
        if gpu:
            x_cat = x_cat.cuda()
            price = price.cuda()

        y_hat = model(x_cat)
        if unsqueeze: y_hat = y_hat.unsqueeze(1)
        loss = F.binary_cross_entropy_with_logits(y_hat,price)
        sum_loss += batch*(loss.item())
        total += batch
    return sum_loss/total

OUTPUT_PATH = '/data/ssubrahmannian/DL/hw2/models/'
def train_loop(model, epochs, optim, lr=0.01, wd=0.0,gpu=False):
    for i in tnrange(epochs,desc='Epochs:',dynamic_ncols=True): 
        train_loss = train_model(model, optim, train_dl,gpu=gpu)
        vld_loss = val_loss(model, valid_dl=vld_dl,gpu=gpu)
        tqdm.write("Training Loss : %f 'Validation Loss: %f"%(np.sqrt(train_loss),np.sqrt(vld_loss)),nolock=True)
        if i%10 ==0: 
            model_output_path = f'{OUTPUT_PATH}model_{i}.pth'
            save_model(model,model_output_path)

def get_optimizer(model,lr,wd=0):
    # Setting optimizer
    parameters = filter(lambda x:x.requires_grad,model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr,weight_decay=wd)
    return optim

def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))


In [23]:
model = CNN()
parameters = filter(lambda x:x.requires_grad,model.parameters())
optim = torch.optim.Adam(parameters, lr=0.01,weight_decay=0)

In [ ]:
train_loop(model,10,optim,gpu=False)

Validation: 100%|██████████| 3/3 [00:03<00:00,  1.07s/it]


Training RMSLE : 0.834441 'Validation RMSLE: 0.832555


Training:   0%|          | 0/8 [00:00<?, ?it/s]

Training RMSLE : 0.832555 'Validation RMSLE: 0.832555


Training:  62%|██████▎   | 5/8 [09:54<05:56, 118.99s/it]